In [ ]:
"""
Analyze the results from the validation testing

"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path
import neuron_visualizations as nviz


import datajoint_utils as du
from importlib import reload

INFO - 2020-12-01 00:47:56,551 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-01 00:47:56,552 - settings - Setting database.user to celiib
INFO - 2020-12-01 00:47:56,553 - settings - Setting database.password to newceliipass
INFO - 2020-12-01 00:47:56,556 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-01 00:47:56,557 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-01 00:47:56,571 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-12-01 00:47:56,782 - settings - Setting enable_python_native_blobs to True


In [2]:
import time
import datajoint_utils as du
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

from datajoint_utils import *

import error_detection as ed

minnie,schema = du.configure_minnie_vm()

INFO - 2020-12-01 00:48:14,668 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-01 00:48:14,870 - settings - Setting enable_python_native_blobs to True


# Results stored in the table

In [12]:
minnie.ManualProofreadValidationStats()

segment_id segment id for those to be decimated,new_segment_id,true_positive_presyn,false_positive_presyn,true_negative_presyn,false_negative_presyn,precision_presyn,recall_presyn,f1_presyn,true_positive_postsyn,false_positive_postsyn,true_negative_postsyn,false_negative_postsyn,precision_postsyn,recall_postsyn,f1_postsyn,true_positive_total,false_positive_total,true_negative_total,false_negative_total,precision_total,recall_total,f1_total
864691134988386682,864691135278269857,35,0,8,11,1.0,0.7608695652173914,0.8641975308641976,0,0,1861,9,nan,0.0,nan,35,0,1869,20,1.0,0.6363636363636364,0.7777777777777778
864691135113167769,864691135367058169,78,2,14,18,0.975,0.8125,0.8863636363636362,5,156,2032,4,0.031055900621118012,0.5555555555555556,0.058823529411764705,83,158,2046,22,0.34439834024896265,0.7904761904761904,0.4797687861271676
864691135212632448,864691135446675028,254,2,14,15,0.9921875,0.9442379182156134,0.9676190476190475,7,9,3335,3,0.4375,0.7,0.5384615384615384,261,11,3349,18,0.9595588235294118,0.9354838709677419,0.9473684210526315
864691135212690816,864691135775906989,0,0,16,1,nan,0.0,nan,0,0,2334,1,nan,0.0,nan,0,0,2350,2,nan,0.0,nan
864691135233108569,864691136023889209,0,0,3,1,nan,0.0,nan,0,0,730,2,nan,0.0,nan,0,0,733,3,nan,0.0,nan
864691135272164113,864691136194298600,61,0,20,9,1.0,0.8714285714285714,0.9312977099236641,0,11,5636,18,0.0,0.0,nan,61,11,5656,27,0.8472222222222222,0.6931818181818182,0.7625000000000001
864691135348272855,864691136143561524,342,0,12,5,1.0,0.9855907780979827,0.9927431059506531,4,31,5967,20,0.11428571428571428,0.16666666666666666,0.13559322033898305,346,31,5979,25,0.9177718832891246,0.9326145552560647,0.9251336898395722
864691135382947307,864691135593659947,13,0,9,0,1.0,1.0,1.0,1,34,2591,0,0.02857142857142857,1.0,0.05555555555555556,14,34,2600,0,0.2916666666666667,1.0,0.45161290322580644
864691135393964789,864691137019596142,133,0,14,68,1.0,0.6616915422885572,0.7964071856287425,11,0,1592,96,1.0,0.102803738317757,0.1864406779661017,144,0,1606,164,1.0,0.4675324675324675,0.6371681415929203
864691135407289801,864691135848030814,184,0,9,4,1.0,0.9787234042553191,0.989247311827957,4,203,3073,3,0.01932367149758454,0.5714285714285714,0.037383177570093455,188,203,3082,7,0.48081841432225064,0.9641025641025641,0.6416382252559727


# Computing the Stats

In [4]:
import statistics_utils as st_u

In [5]:
TP,FP,FN = minnie.ManualProofreadValidationStats.fetch("true_positive_total","false_positive_total","false_negative_total")
total_dict = dict(type="presyn and postsyn")
total_stats = st_u.calculate_scores(TP = np.sum(TP),FP = np.sum(FP),FN = np.sum(FN))
total_dict.update(total_stats)
total_dict

{'type': 'presyn and postsyn',
 'precision': 0.5836415947390053,
 'recall': 0.7924107142857143,
 'f1': 0.6721893491124261}

In [17]:
np.sum(TP_pre), np.sum(TP_pre) + np.sum(FN_pre)

(3374, 3808)

In [13]:
TP_pre,FP_pre,FN_pre = minnie.ManualProofreadValidationStats.fetch("true_positive_presyn","false_positive_presyn","false_negative_presyn")
presyn_dict = dict(type="presyn")
presyn_stats = st_u.calculate_scores(TP = np.sum(TP_pre),FP = np.sum(FP_pre),FN = np.sum(FN_pre))
presyn_dict.update(presyn_stats)
presyn_dict

{'type': 'presyn',
 'precision': 0.9759907434191496,
 'recall': 0.8860294117647058,
 'f1': 0.9288368891947695}

In [7]:
TP_post,FP_post,FN_post = minnie.ManualProofreadValidationStats.fetch("true_positive_postsyn","false_positive_postsyn","false_negative_postsyn")
postsyn_dict = dict(type="postsyn")
postsyn_stats = st_u.calculate_scores(TP = np.sum(TP_post),FP = np.sum(FP_post),FN = np.sum(FN_post))
postsyn_dict.update(postsyn_stats)
postsyn_dict

{'type': 'postsyn',
 'precision': 0.042735042735042736,
 'recall': 0.17716535433070865,
 'f1': 0.06885998469778118}

# Putting the Results in a Nice Table

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()

In [ ]:
new_dic = df.append([presyn_dict,postsyn_dict,total_dict],ignore_index=True)
new_dic

In [ ]:
filtered_columns  = new_dic[["precision","recall","f1"]]#
accuracies = filtered_columns.to_numpy()
accuracies

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

#setting the text size
matplotlib.rcParams.update({'font.size': 17})


cm = accuracies
normalize = True
cmap=plt.cm.Blues
#now graph the results for all the categories
title = "Autoproofreading Validation"

fig, ax = plt.subplots()
im = ax.imshow(accuracies, interpolation='nearest', cmap=cmap)
ax.figure.colorbar(im, ax=ax)

ax.set(xticks=np.arange(accuracies.shape[1]),
       yticks=np.arange(accuracies.shape[0]),
       # ... and label them with the respective list entries
       xticklabels=["Inputs","Outputs","Both"], yticklabels=["precision","recall","f1"],
       title=title,
       ylabel='Statistics',
       xlabel='Type')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = '.2f' if normalize else 'd'
thresh = np.nanmax(cm) / 2.
thresh = 0.6
#print("threshold = " + str(thresh))
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        #print("cm[i,j] = " + str(cm[i,j]))
        ax.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black",
               fontsize=20)
fig.tight_layout()

fig.set_size_inches(20, 12)
ax.grid(False)

# Computing the Precision and Recall for everyone

In [ ]:
minnie.ValidationStats()

In [ ]:
filtered_table = (minnie.ValidationStats() & "recall_presyn > 0" & "recall_postsyn > 0" & "recall_total > 0")
filtered_table

In [ ]:
"""
Pseudocode: 
1) Get the precision and recall for all of the neurons 
2) Filter away those that are Nan
3) Plot the precision and recall distribution
4) Get the median precision and recall
"""

In [ ]:
#1) Get the precision and recall for all of the neurons 
(precision_presyn,precision_postsyn,precision_total,
 recall_presyn,recall_postsyn,recall_total) = filtered_table.fetch("precision_presyn","precision_postsyn","precision_total",
                                                                  "recall_presyn","recall_postsyn","recall_total")

In [ ]:
precision_dicts = [dict(stat="precision",inputs=pre,ouptuts=post,both=b) for pre,post,b in zip(precision_presyn,precision_postsyn,precision_total)]
recall_dicts = [dict(stat="recall",inputs=pre,ouptuts=post,both=b) for pre,post,b in zip(recall_presyn,recall_postsyn,recall_total)]
total_dicts=  precision_dicts + recall_dicts

In [ ]:
input_info = [precision_presyn,recall_presyn]
output_info = [precision_postsyn,recall_postsyn]
both_info = [precision_total,recall_total]
total_info = [input_info,output_info,both_info]

info_name = ["input","output","both"]

revised_dicts= []
for i_name,(prec,rec) in zip(info_name,total_info):
    for p,r in zip(prec,rec):
        revised_dicts.append({"Type":i_name,"Precision":p,"Recall":r})
revised_dicts

# Plotting the Precisions

In [ ]:
import seaborn as sns
import pandas as pd

# create a dataframe to pass to it

In [ ]:
df = pd.DataFrame()
new_dict = df.append(revised_dicts)
new_dict

In [ ]:
plt.figure(figsize=(20,20))
sns.set(rc={'figure.figsize':(20,8.27)})

g = sns.FacetGrid(new_dict, col="Type")
g.map(sns.scatterplot, "Precision", "Recall", alpha=.7)
g.add_legend()

plt.show()

In [ ]:
collapsed_df = new_dict.groupby("Type").median()

In [ ]:
reordered_medians = collapsed_df.iloc[[1,2,0]]
median_accuracies = reordered_medians.to_numpy()
median_accuracies.T

In [ ]:
median_accuracies.T

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

#setting the text size


accuracies = median_accuracies.T
cm = accuracies
normalize = True
cmap=plt.cm.Blues
#now graph the results for all the categories
title = "Autoproofreading Validation"

fig, ax = plt.subplots()
im = ax.imshow(accuracies, interpolation='nearest', cmap=cmap)
ax.figure.colorbar(im, ax=ax)

ax.set(xticks=np.arange(accuracies.shape[1]),
       yticks=np.arange(accuracies.shape[0]),
       # ... and label them with the respective list entries
       xticklabels=["Inputs","Outputs","Both"], yticklabels=["precision","recall"],
       title=title,
       ylabel='Statistics (Median)',
       xlabel='Type')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = '.2f' if normalize else 'd'
thresh = np.nanmax(cm) / 2.
thresh = 0.6
#print("threshold = " + str(thresh))
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        #print("cm[i,j] = " + str(cm[i,j]))
        ax.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black",
               fontsize=15)
fig.tight_layout()
plt.rcParams.update({'font.size': 2})
fig.set_size_inches(7, 4)
ax.grid(False)

# Histogram of Per Neuron what we are removing

In [ ]:
n_errored_synapses = (minnie.AutoProofreadLabels & "n_synapses>0").fetch("n_errored_synapses")

In [ ]:
np.max(n_errored_synapses)

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1)
ax.hist(n_errored_synapses,bins=1000,density=False)
ax.set_title("Number of Synapses Removed Per Neuron \n (Synapses Restricted to Direct Connection Matrix)")
ax.set_xlabel("Number of Synapses Removed")
ax.set_ylabel("Counts")

# ax.set_yscale("log")
#ax.set_xscale("log")
plt.show()